<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN07/7.0-Netzwerktopologien_Best_Practises_pl.ipynb)

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# Bewährte Verfahren des Deep Learnings (PyTorch Lightning Edition)

Diese Notebooks zeigen die in Kapitel 6, Abschnitt 3 von [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff) gezeigten Konzepte, angepasst für PyTorch Lightning.
# 
Die Themen:
- Die **funktionale Lightning-API** (via `nn.Module` und `LightningModule`)
- **Callbacks** und Lightning's `Trainer`
- TensorBoard für Visualisierung
- Praktiken wie *Batch Normalization*, *Residual Connections*, *Hyperparameter Tuning*, *Ensembles*

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L

print(f"PyTorch Version: {torch.__version__}")
print(f"Lightning Version: {L.__version__}")


In dieser Lektion lernen wir eine Reihe leistungsfähiger Tools kennen, die es uns erleichtern, schwierige Aufgaben Modelle nach dem aktuellen Stand der Technik zu entwickeln. Mit der funktionalen Lightning-API können wir graphenähnliche Modelle entwickeln, einem Layer verschiedene Eingaben übergeben und Lightning-Modelle wie Python-Funktionen verwenden.
- Lightning-Callbacks und das browserbasierte Visualisierungstool TensorBoard ermöglichen es, Modelle während des Trainings zu überwachen.
- Darüber hinaus kommen verschiedene andere bewährte Verfahren zur Sprache, wie die Normierung von Stapeln (*batch normalization*), residuale Verbindungen (*residual connections*), Hyperparameteroptimierung (*hyperparameter tuning*) und Ensemblemodelle (*ensemble models*).
### 7.1 Jenseits des Sequential-Modells: die funktionale Lightning-API
Bislang wurden die meisten vorgestellten Modelle mithilfe des Sequential-Modells implementiert. Diesem Modell liegen die Annahmen zugrunde, dass das NN genau eine Eingabe erhält, genau eine Ausgabe liefert und aus einem linearen Stapel von Layern besteht.

Diese Annahmen haben sich schon häufig als richtig erwiesen. Der Aufbau ist tatsächlich so gebräuchlich, dass wir in der Lage waren, eine Vielzahl verschiedener Themen und praktischer Anwendungen zu erörtern, und dabei bisher ausschließlich die Sequential-Klasse verwendet haben. Allerdings sind diese Annahmen in manchen Fällen nicht flexibel genug. Bei einigen NNs sind mehrere Eingaben erforderlich, andere benötigen mehrere Ausgaben, und wieder andere besitzen interne Verzweigungen zwischen den Layern und sehen deshalb nicht mehr wie ein linearer Stapel, sondern wie ein Graph von Layern aus.
### Modelle mit multimodalen Eingängen

Bei manchen Aufgaben sind beispielsweise multimodale Eingaben erforderlich: Daten aus mehreren Eingabequellen werden zusammengeführt, wobei unterschiedliche Layer die verschiedenen Datentypen verarbeiten. 

Stellen Sie sich ein Deep-Learning-Modell vor, das versucht, den wahrscheinlichsten Preis eines Kleidungsstücks aus zweiter Hand vorherzusagen, und zu diesem Zweck die folgenden Eingaben verwendet: 
- vom Benutzer bereitgestellte *Metadaten* (wie Marke, Alter, Grösse usw.), 
- eine vom Benutzer bereitgestellte *Beschreibung in Textform* sowie 
- ein *Foto* des Kleidungsstücks.

Wenn nur die Metadaten verfügbar wären, könnten Sie die *One-hot-Codierung* verwenden und mit einem Fully-connected NN den Preis vorhersagen. Wenn nur die Beschreibung in Textform vorläge, könnten Sie ein RNN oder ein 1-D-CNN verwenden. Und wenn Sie nur das Foto hätten, könnten Sie ein 2-D-CNN verwenden. 

Aber wie kann man alles gleichzeitig nutzen?
Ein naiver Ansatz wäre es, drei verschiedene Modelle zu trainieren und anschliessend einen gewichteten Mittelwert der Vorhersagen zu berechnen. Diese Vorgehensweise könnte jedoch suboptimal sein, weil die von den Modellen extrahierten Informationen womöglich *redundant* sind. Besser ist es, dem Modell alle drei Eingaben gleichzeitig bereitzustellen, sodass durch das *Zusammenwirken der verschiedenen Eingaben* ein präziseres Modell der Daten erlernt werden kann. Auf diese Weise entsteht ein Modell mit drei Eingabezweigen.

<img src="Bilder/MultiInputModell.png" width="640"  align="center"/>

In einigen Fällen kann es auch erforderlich sein, mehrere **Zielattribute** anhand der Eingabedaten vorherzusagen. 
- Vielleicht möchten Sie den Text eines Romans oder einer Kurzgeschichte automatisch nach Genre (wie z.B. Romanze oder Thriller) klassifizieren, aber gleichzeitig auch den ungefähren Zeitpunkt vorhersagen, zu  dem der Text geschrieben wurde. Natürlich könnten Sie zwei verschiedene Modelle trainieren: eins für das Genre und ein zweites für den Zeitpunkt, aber da diese beiden Attribute statistisch nicht voneinander unabhängig sind, können Sie ein besseres Modell entwickeln, das erlernt, Genre und Zeitpunkt gleichzeitig vorherzusagen.

Ein solches Modell würde **zwei Ausgaben** liefern. Aufgrund der Korrelationen zwischen Genre und Zeitpunkt würde die Kenntnis des Zeitpunkts der Entstehung eines Romans dem Modell dabei helfen, ergiebige und genaue Repräsentationen im Raum der Romangenres zu erlernen. Umgekehrt wäre auch die Kenntnis des Genres zur Vorhersage des Zeitpunkts nützlich.

<img src="Bilder/MultiOutputModell.png" width="640"  align="center"/>

### Nichtlineare Netztopologien (DAG)

Darüber hinaus benötigen viele der in jüngster Zeit entwickelten Architekturen **nicht-lineare Netztopologien**: 
- Netze, die die Form eines gerichteten *azyklischen Graphen* besitzen. 
- Die von *Szegedy et al.*[1] bei Google entwickelte Inceptions-Familie neuronaler Netze beispielsweise beruht auf Inception-Modulen, bei denen Eingaben von mehreren parallel verlaufenden Faltungszweigen verarbeitet werden, deren Ausgaben wieder zu einem einzelnen Tensor zusammengeführt werden.

<img src="Bilder/InceptionModule.png" width="640"  align="center"/>

[1] [Christian Szegedy et al., Going Deeper with Convolutions, Conference on Computer Vision and
Pattern Recognition (2014)](https://arxiv.org/abs/1409.4842).

### Residual Connections

Und dann gibt es noch den Trend, Modellen **residuale Verbindungen** hinzuzufügen, der mit der von *He et al.*[2] bei Microsoft entwickelten **ResNet-Familie** neuronaler Netze einsetzte. Bei einer residualen Verbindung werden frühere Repräsentationen dem nachfolgenden Datenstrom wieder hinzugefügt, indem ein älterer Ausgabetensor zu einem jüngeren Ausgabetensor hinzuaddiert wird. Dadurch lässt sich ein Informationsverlust während des Ablaufs der Datenverarbeitung verhindern. Für diese graphenähnlichen Netze gibt es viele weitere Beispiele.

<img src="Bilder/ResidualModule.png" width="440"  align="center"/>


[2] [Kaiming He et al., Deep Residual Learning for Image Recognition, Conference on Computer
Vision and Pattern Recognition (2015)](https://arxiv.org/abs/1512.03385).

### Lightning functional API
Diese drei wichtigen Anwendungsfälle (Modelle mit mehreren Eingaben, Modelle mit mehreren Ausgaben sowie graphenähnliche Modelle) sind in Lightning nicht realisierbar, wenn nur die Sequential-Klasse verwendet wird.

Es gibt jedoch eine weitere, sehr viel allgemeinere und flexiblere Möglichkeit, Lightning zu nutzen: die funktionale API. Der folgende Abschnitt erklärt, was genau das ist, was sie leistet und wie man sie verwendet.

Mit der funktionalen API können Sie Tensoren direkt bearbeiten und Layer wie Funktionen verwenden, die Tensoren entgegennehmen und zurückliefern (daher auch die Bezeichnung funktionale API).

In [ ]:
from torch import nn
import torch

# Der Input ist Tensor
input_tensor = torch.randn(1, 32)
# Ein Layer ist eine Funktion.
dense = nn.Linear(32, 32)
# Ein Layer kann mit einem Tensor aufgerufen werden und gibt einen Tensor zurück
output_tensor = torch.relu(dense(input_tensor))

print(output_tensor)


Wir definieren ein einfaches Modell mit drei linearen Schichten als ``LightningModule`` und verschaffen uns mit torchsummary einen Überblick über die Architektur.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torchsummary import summary

# Definiere ein einfaches Modell als LightningModule
class SimpleModel(L.LightningModule):
    def __init__(self):
        super(SimpleModel, self).__init__()
        # Definiere die Layer des Modells
        self.layer1 = nn.Linear(64, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, 10)
        # Listen zur Speicherung der Trainingsverluste
        self.train_losses = []
        self._epoch_losses = []  # Temporäre Speicherung pro Batch

    def forward(self, x):
        # Definiere den Vorwärtsdurchlauf
        x = F.relu(self.layer1(x))  # Aktivierungsfunktion ReLU nach dem ersten Layer
        x = F.relu(self.layer2(x))  # Aktivierungsfunktion ReLU nach dem zweiten Layer
        x = self.layer3(x)  # Ausgabe des dritten Layers
        return x

# Überprüfe, ob eine GPU verfügbar ist, und verwende sie, falls möglich
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialisiere das Modell und verschiebe es auf das entsprechende Gerät (CPU oder GPU)
model = SimpleModel().to(device)

# Zeige eine Zusammenfassung des Modells an, wobei die Eingabegröße (64,) ist
summary(model, (64,))


Das Einzige, was an dieser Stelle ein wenig wie Zauberei wirkt, ist die **Instanziierung eines Model-Objekts allein durch Übergabe eines Eingabe- und eines Ausgabetensors**.
Hinter den Kulissen ruft Lightning sämtliche Layer ab, die daran beteiligt
sind, aus dem Eingabetensor `input_tensor` den Ausgabetensor `output_tensor`
zu berechnen, und fasst sie in einer graphenähnlichen Datenstruktur zusammen – einem Modell. 

Das Ganze funktioniert natürlich nur, weil `output_tensor` durch wiederholte Transformation von `input_tensor` entstanden ist. Wenn wir versuchen, ein Modell mit Ein- und Ausgaben zu erzeugen, die nicht zusammengehören, löst das einen Laufzeitfehler aus:

In [ ]:
class BadModel(L.LightningModule):
    def __init__(self):
        super(BadModel, self).__init__()
        # Definiere die Layer des Modells
        self.layer1 = nn.Linear(64, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, 10)

    def forward(self, x):
        # Definiere den Vorwärtsdurchlauf mit ReLU-Aktivierungen
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

# Initialisiere das Modell
bad_model = BadModel()

# Definieren Sie den unrelated_input Tensor
# Dieser Tensor hat eine falsche Eingabegröße, um einen Fehler zu verursachen
unrelated_input = torch.randn(1, 32)

# Versuch, ein Modell mit nicht zusammengehörenden Eingabe- und Ausgabetensoren zu erstellen
try:
    # Dies wird einen Laufzeitfehler verursachen, da die Eingabegröße nicht mit der erwarteten Größe übereinstimmt
    output = bad_model(unrelated_input)
except RuntimeError as e:
    # Fange den Laufzeitfehler ab und drucke die Fehlermeldung
    print(f"Laufzeitfehler: {e}")


Die Fehlermeldung weist darauf hin, dass es PyTorch Lightning nicht gelungen ist, den Ausgabetensor des Modells korrekt auf den erwarteten Eingabetensor input_1 abzubilden. Dies tritt typischerweise auf, wenn die Shapes oder die Datenformate der Ein- und Ausgaben nicht zueinander passen oder nicht korrekt transformiert wurden.

In diesem Fall verwenden wir ein benutzerdefiniertes Modell, das auf der LightningModule-Klasse basiert. Dabei definieren wir explizit den Vorwärtsdurchlauf, die Verlustberechnung und die Optimierung. Es liegt also in unserer Verantwortung, sicherzustellen, dass die Ein- und Ausgabetensoren korrekt verarbeitet und miteinander kompatibel sind.

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import lightning as L
import torch.nn.functional as F
import matplotlib.pyplot as plt

# Definiere die Trainingsschritt-Funktion
def training_step(self, batch, batch_idx):
    x, y = batch  # Entpacke den Batch in Eingaben (x) und Zielwerte (y)
    y_hat = self(x)  # Berechne die Vorhersagen des Modells
    loss = F.cross_entropy(y_hat, y)  # Berechne den Cross-Entropy-Verlust
    self.log("train_loss", loss, prog_bar=True)  # Logge den Verlust für die Fortschrittsanzeige
    self._epoch_losses.append(loss.item())  # Füge den Verlust zur Liste der Verluste pro Epoche hinzu
    return loss  # Gib den Verlust zurück

# Definiere die Funktion, die am Ende jeder Epoche aufgerufen wird
def on_train_epoch_end(self):
    if self._epoch_losses:  # Überprüfe, ob die Liste der Verluste pro Epoche nicht leer ist
        avg_loss = sum(self._epoch_losses) / len(self._epoch_losses)  # Berechne den durchschnittlichen Verlust
        self.train_losses.append(avg_loss)  # Füge den durchschnittlichen Verlust zur Liste der Trainingsverluste hinzu
        self._epoch_losses.clear()  # Leere die Liste der Verluste pro Epoche für die nächste Epoche

# Definiere die Optimierer-Konfigurationsfunktion
def configure_optimizers(self):
    return torch.optim.RMSprop(self.parameters(), lr=0.01)  # Verwende RMSprop als Optimierer mit einer Lernrate von 0.01

# Füge die definierten Methoden zur SimpleModel-Klasse hinzu
SimpleModel.training_step = training_step
SimpleModel.on_train_epoch_end = on_train_epoch_end
SimpleModel.configure_optimizers = configure_optimizers

# Dummy-Daten generieren und normalisieren
x_train = np.random.random((1000, 64)).astype(np.float32)  # Generiere zufällige Trainingsdaten
y_train = np.random.randint(0, 10, (1000,)).astype(np.int64)  # Generiere zufällige Zielwerte
x_train = (x_train - np.mean(x_train, axis=0)) / np.std(x_train, axis=0)  # Normalisiere die Trainingsdaten

# In Tensoren umwandeln
x_train_tensor = torch.tensor(x_train)  # Konvertiere die Trainingsdaten in einen Tensor
y_train_tensor = torch.tensor(y_train)  # Konvertiere die Zielwerte in einen Tensor

# DataLoader erstellen
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)  # Erstelle ein Dataset aus den Tensoren
train_loader = DataLoader(train_dataset, batch_size=32)  # Erstelle einen DataLoader mit einer Batch-Größe von 32

# Modell und Trainer initialisieren
model = SimpleModel()  # Initialisiere das Modell
trainer = L.Trainer(max_epochs=30, logger=False, enable_checkpointing=False)  # Initialisiere den Trainer mit 30 Epochen
trainer.fit(model, train_loader)  # Trainiere das Modell mit dem DataLoader

# Lernkurve plotten
plt.plot(model.train_losses, marker="o")  # Plotte die Trainingsverluste
plt.xlabel("Epoche")  # Beschrifte die x-Achse
plt.ylabel("Cross Entropy Loss")  # Beschrifte die y-Achse
plt.title("Lernkurve")  # Setze den Titel des Plots
plt.grid(True)  # Zeige ein Gitter im Plot an
plt.show()  # Zeige den Plot an


## 7.1 Modelle mit mehreren Eingaben

Mit der funktionalen API können Modelle mit mehreren Eingaben erstellt werden. Typischerweise führen Modelle dieser Art die verschiedenen Eingabezweige mit einem Layer zusammen, der mehrere Tensoren miteinander kombiniert: durch **Addition, durch Verkettung usw**.

Für gewöhnlich wird diese Aufgabe mittels PyTorch-Operationen wie `torch.add` und `torch.cat` erledigt. Betrachten wir ein besonders einfaches Beispiel für ein Modell mit mehreren Eingaben: ein Modell zum Beantworten von Fragen.

Ein solches Modell besitzt typischerweise zwei Eingaben: eine in natürlicher Sprache formulierte Frage und einen Textabschnitt (z.B. einen Zeitungsartikel), der die Informationen zum Beantworten der Frage bereitstellt. Das Modell muss nun eine Antwort geben. Im einfachsten Fall besteht die Antwort aus einem einzelnen Wort, das via softmax-Funktion anhand eines vorgegebenen Vokabulars ermittelt wird.

<img src="Bilder/ChatBot.png" width="440" align="center"/>

Das folgende Beispiel zeigt, wie Sie mit der funktionalen API ein solches Modell
erstellen können. Sie richten zwei voneinander unabhängige Zweige ein, codieren
den Text der möglichen Antworten und die Frage als Repräsentationsvektoren, verketten
diese Vektoren und fügen den verketteten Repräsentationen schließlich
einen softmax-Klassifizierer hinzu.

In [ ]:
import lightning as L
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from torchviz import make_dot

# Dummy data configuration
num_samples = 1000
max_length = 100
text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# Dummy data generation
text = torch.randint(1, text_vocabulary_size, (num_samples, max_length))
question = torch.randint(1, question_vocabulary_size, (num_samples, max_length))
answers = torch.randint(0, answer_vocabulary_size, (num_samples,))

# Create a dataset and dataloader
dataset = TensorDataset(text, question, answers)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Define the model class
class TextQuestionModel(L.LightningModule):
    def __init__(self, text_vocab_size, question_vocab_size, answer_vocab_size):
        super(TextQuestionModel, self).__init__()
        # Embedding layers for text and question inputs
        self.text_embedding = nn.Embedding(text_vocab_size, 64)
        self.text_lstm = nn.LSTM(64, 32, batch_first=True)
        self.question_embedding = nn.Embedding(question_vocab_size, 32)
        self.question_lstm = nn.LSTM(32, 16, batch_first=True)
        # Fully connected layer to combine the outputs of the LSTMs
        self.fc = nn.Linear(32 + 16, answer_vocab_size)
        # Softmax activation for the output
        self.softmax = nn.Softmax(dim=-1)
        # Loss function
        self.criterion = nn.CrossEntropyLoss()

    def forward(self, text, question):
        # Forward pass for text input
        embedded_text = self.text_embedding(text)
        _, (encoded_text, _) = self.text_lstm(embedded_text)
        # Forward pass for question input
        embedded_question = self.question_embedding(question)
        _, (encoded_question, _) = self.question_lstm(embedded_question)
        # Concatenate the last hidden states of the LSTMs from the two different inputs
        concatenated = torch.cat((encoded_text[-1], encoded_question[-1]), dim=-1)
        # Pass the concatenated tensor through the fully connected layer
        logits = self.fc(concatenated)
        return logits

    def training_step(self, batch, batch_idx):
        # Unpack the batch
        text, question, answers = batch
        # Forward pass
        logits = self.forward(text, question)
        # Compute the loss
        loss = self.criterion(logits, answers)
        # Compute the accuracy
        preds = logits.argmax(dim=-1)
        acc = (preds == answers).float().mean()
        # Log the loss and accuracy
        self.log("train_loss", loss)
        self.log("train_acc", acc)
        return loss

    def configure_optimizers(self):
        # Use RMSprop optimizer
        return optim.RMSprop(self.parameters(), lr=0.001)

# Initialize the model
model = TextQuestionModel(
    text_vocabulary_size, question_vocabulary_size, answer_vocabulary_size
)

# Initialize the trainer
trainer = L.Trainer(max_epochs=10)
# Train the model
trainer.fit(model, dataloader)

# Visualize and save the model architecture
sample_text = torch.randint(1, text_vocabulary_size, (1, max_length))
sample_question = torch.randint(1, question_vocabulary_size, (1, max_length))
logits = model(sample_text, sample_question)
dot = make_dot(logits, params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("TextQuestionModel_architecture", format="png")


In [ ]:
from IPython.display import Image, display
display(Image(filename='TextQuestionModel_architecture.png'))


Um dieses Modell mit zwei Eingaben zu trainieren, gibt es zwei mögliche APIs:

- Sie können dem Modell eine Liste von PyTorch-Tensoren oder ein Dictionary übergeben, das den Eingabebezeichnungen PyTorch-Tensoren zuordnet. Die letztere Option ist natürlich nur verfügbar, wenn Sie die Eingaben benannt haben.

## 7.2 Modelle mit mehreren Ausgaben

Auch in PyTorch Lightning lässt sich auf einfache Weise ein Modell mit mehreren Ausgaben erstellen. Als Beispiel betrachten wir ein neuronales Netz, das mehrere Eigenschaften der Daten gleichzeitig vorhersagen soll. Man könnte sich etwa ein System vorstellen, das eine Reihe von Social-Media-Beiträgen als Eingabe erhält und daraus verschiedene Merkmale einer Person schätzt, z.B. Alter, Geschlecht oder Einkommen.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as L
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from torchviz import make_dot

# Dummy-Datenkonfiguration
num_samples = 1000
max_length = 100
vocabulary_size = 50000
num_income_groups = 10

# Dummy-Daten generieren
posts = torch.randint(1, vocabulary_size, (num_samples, max_length))
ages = torch.randint(0, 100, (num_samples,))
incomes = torch.randint(0, num_income_groups, (num_samples,))
genders = torch.randint(0, 2, (num_samples,))

# Dataset und DataLoader erstellen
dataset = TensorDataset(posts, ages, incomes, genders)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

# Definiere das Modell mit mehreren Ausgaben
class MultiOutputModel(L.LightningModule):
    def __init__(self, vocabulary_size, num_income_groups):
        super(MultiOutputModel, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, 256)
        self.conv1 = nn.Conv1d(256, 128, 3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(128, 256, 3)
        self.conv3 = nn.Conv1d(256, 256, 3)
        self.pool2 = nn.MaxPool1d(2)
        self.conv4 = nn.Conv1d(256, 256, 3)
        self.conv5 = nn.Conv1d(256, 256, 3)
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(256, 128)
        self.age_output = nn.Linear(128, 1)
        self.income_output = nn.Linear(128, num_income_groups)
        self.gender_output = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.global_pool(x).squeeze(-1)
        x = F.relu(self.fc1(x))
        age = self.age_output(x)
        income = self.income_output(x)
        gender = torch.sigmoid(self.gender_output(x))
        return age, income, gender

    def training_step(self, batch, batch_idx):
        posts, ages, incomes, genders = batch
        age_pred, income_pred, gender_pred = self(posts)
        age_loss = F.mse_loss(age_pred.squeeze(), ages.float())
        income_loss = F.cross_entropy(income_pred, incomes)
        gender_loss = F.binary_cross_entropy(gender_pred.squeeze(), genders.float())
        loss = age_loss + income_loss + gender_loss
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)

# Modell und Trainer initialisieren
model = MultiOutputModel(vocabulary_size, num_income_groups)
trainer = L.Trainer(max_epochs=10)
trainer.fit(model, dataloader)

# Modellarchitektur visualisieren und speichern
sample_input = torch.randint(1, vocabulary_size, (1, max_length))
age, income, gender = model(sample_input)
dot = make_dot((age, income, gender), params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("MultiOutputModel_architecture", format="png")


 An dieser Stelle ist es wichtig zu beachten, dass ein solches Modell die Moeglichkeit bieten muss, fuer die verschiedenen Ausgaben unterschiedliche Verlustfunktionen zu verwenden.

- Die Vorhersage des Alters ist beispielsweise eine skalare Regression,
- die Vorhersage des Geschlechts hingegen eine Binaerklassifikation, die eine andere Verlustfunktion erfordert.

Da beim Gradientenabstiegsverfahren jedoch ein einzelner Skalar minimiert wird, muessen die verschiedenen Verlustfunktionen zu einem gemeinsamen Gesamtverlust kombiniert werden. Die einfachste Methode besteht darin, die Werte der einzelnen Verluste einfach zu summieren.

In PyTorch Lightning erfolgt diese Kombination direkt in der Methode ``training_step``, wo man die jeweiligen Verluste berechnet und selbst addiert. Lightning erwartet, dass du am Ende einen Gesamtverlustwert zurueckgibst, der dann vom Framework optimiert wird.

Die Funktion configure_optimizers dient in Lightning ausschliesslich dazu, Optimierer (und optional Lernraten-Scheduler) zu definieren – nicht aber zur Verwaltung oder Kombination von Verlustfunktionen.

In [ ]:
from IPython.display import Image, display
# Replace 'example.png' with the path to your PNG file.
display(Image(filename='MultiOutputModel_architecture.png'))


<div class="alert alert-block alert-info">

### Gewichtung
Beachten Sie hier, dass sehr unausgewogene Beiträge zum Verlust dazu führen können, dass die Repräsentationen des Modells bevorzugt für die Aufgabe mit den größten Werten der Verlustfunktion optimiert werden – und zwar auf Kosten der übrigen Aufgaben.

- Um das zu verhindern, können Sie die Beiträge gewichten, die die verschiedenen Werte der Verlustfunktion zum Gesamtverlust leisten.
- Das ist besonders nützlich, wenn die Werte der unterschiedlichen Verlustfunktionen in verschiedenen Größenordnungen liegen.
- Bei der Regression zur Altersvorhersage beispielsweise nimmt der mittlere quadratische Fehler (mse) typischerweise Werte zwischen ca. 3 und 5 an, während der Wert der für die Geschlechtsklassifikation verwendeten Kreuzentropie manchmal nur 0.1 beträgt.
- Um ausgeglichenere Beiträge der verschiedenen Verluste zu erzielen, können Sie z.B. der Kreuzentropie eine Gewichtung von 10 und dem mittleren quadratischen Fehler eine Gewichtung von 0.25 zuweisen.

In PyTorch Lightning erfolgt diese Gewichtung direkt im training_step, indem Sie die einzelnen Verluste mit entsprechenden Faktoren multiplizieren, bevor sie zum Gesamtverlust aufsummiert werden.

</div>

In [ ]:
class MultiOutputModel(L.LightningModule):
    def __init__(self, vocabulary_size, num_income_groups):
        super(MultiOutputModel, self).__init__()
        self.embedding = nn.Embedding(vocabulary_size, 256)
        self.conv1 = nn.Conv1d(256, 128, 3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(128, 256, 3)
        self.conv3 = nn.Conv1d(256, 256, 3)
        self.pool2 = nn.MaxPool1d(2)
        self.conv4 = nn.Conv1d(256, 256, 3)
        self.conv5 = nn.Conv1d(256, 256, 3)
        self.global_pool = nn.AdaptiveMaxPool1d(1)
        self.fc1 = nn.Linear(256, 128)
        self.age_output = nn.Linear(128, 1)
        self.income_output = nn.Linear(128, num_income_groups)
        self.gender_output = nn.Linear(128, 1)

    def forward(self, x):
        x = self.embedding(x).permute(0, 2, 1)
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = self.pool2(x)
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = self.global_pool(x).squeeze(-1)
        x = F.relu(self.fc1(x))
        age = self.age_output(x)
        income = self.income_output(x)
        gender = torch.sigmoid(self.gender_output(x))
        return age, income, gender

    def training_step(self, batch, batch_idx):
        posts, ages, incomes, genders = batch
        age_pred, income_pred, gender_pred = self(posts)
        age_loss = F.mse_loss(age_pred.squeeze(), ages.float())
        income_loss = F.cross_entropy(income_pred, incomes)
        gender_loss = F.binary_cross_entropy(gender_pred.squeeze(), genders.float())

        # Gewichtung der Verluste
        loss = 0.25 * age_loss + 1.0 * income_loss + 10.0 * gender_loss

        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.RMSprop(self.parameters(), lr=0.001)


# Initialisiere das Modell
model = MultiOutputModel(vocabulary_size, num_income_groups)

# Initialisiere den Trainer
trainer = L.Trainer(max_epochs=10)

# Trainiere das Modell mit dem DataLoader
trainer.fit(model, dataloader)


## 7.3 Inception Module: Gerichtete azyklische Graphen von Layern
Mit der flexiblen Struktur von PyTorch und Lightning können Sie nicht nur Modelle mit mehreren Ein- und Ausgaben erstellen, sondern auch neuronale Netze mit komplexer interner Topologie implementieren. PyTorch erlaubt die Umsetzung von Netzen in Form von gerichteten azyklischen Graphen (DAGs).

Azyklisch bedeutet, dass keine Schleifen im Datenfluss erlaubt sind – ein Tensor darf nie zur Eingabe eines Layers werden, der ihn selbst erzeugt hat. Die einzige erlaubte Form von Schleifen betrifft rekurrente Netze, bei denen spezialisierte rekurrente Layer verwendet werden.

Einige bekannte Bausteine neuronaler Netze setzen auf solche Graphenstrukturen – besonders Inception-Module und residuale Verbindungen. Um zu verstehen, wie man solche flexiblen Strukturen in PyTorch Lightning modelliert, betrachten wir ein Beispiel für ein Inception-Modul.

### Inception-Module: Aufbau und Idee
Inception V3 ist eine bekannte Architektur für Convolutional Neural Networks, die 2013/2014 von Christian Szegedy und seinem Team bei Google entwickelt wurde, inspiriert durch das "Network in Network"-Konzept.

Ein Inception-Modul besteht aus mehreren parallel verlaufenden Zweigen, die jeweils unterschiedliche Faltungsoperationen ausführen, und deren Ergebnisse anschließend verkettet werden.

- Die einfachste Variante eines Inception-Moduls hat drei bis vier Zweige.
- Typisch ist eine Kombination aus 1x1-Faltung, gefolgt von 3x3- oder 5x5-Faltungen.
- Abschließend werden die Ergebnisse aller Zweige konkateniert.
- So kann das Netzwerk räumliche und kanalbezogene Merkmale getrennt verarbeiten, was effizienter ist als beides gleichzeitig zu lernen.
Komplexere Varianten integrieren zusätzlich Pooling-Schritte, verschiedene Filtergrößen (z.B. 5x5 statt 3x3), oder reine 1x1-Faltungen ohne räumliche Faltung.

Die folgende Abbildung zeigt ein Beispiel für ein Inception-Modul aus Inception V3:

<img src="Bilder/InceptionModule_V3.png" width="540"  align="center"/>
<div class="alert alert-block alert-warning">

## Die Aufgabe von 1x1-Faltungen in Lightning
Sie wissen bereits, dass Faltungen räumliche Patches aus einem Eingabetensor extrahieren und auf jedes Patch dieselbe Transformation anwenden. Ein Sonderfall tritt bei 1x1-Faltungen auf: Hier besteht jedes Patch nur aus einem einzelnen Feld. In PyTorch bzw. Lightning definieren Sie solche Faltungen mit nn.Conv2d(..., kernel_size=1).

Die Operation wirkt in diesem Fall wie ein Linear-Layer, der auf jedes Feld separat angewendet wird:

- Es werden ausschließlich kanalbezogene Informationen verarbeitet, räumliche Informationen bleiben unberücksichtigt, da jeweils nur ein einzelnes Feld betrachtet wird.
- In Inception-Modulen werden solche 1x1-Faltungen (auch punktweise Faltungen) gezielt eingesetzt, um das Lernen kanalbezogener Merkmale vom Lernen räumlicher Merkmale zu trennen.
- Das ist besonders effizient, wenn man davon ausgeht, dass einzelne Kanäle räumlich stark autokorreliert sind, während zwischen verschiedenen Kanälen kaum Korrelationen bestehen.

In PyTorch Lightning werden 1x1-Faltungen einfach mit nn.Conv2d realisiert und lassen sich flexibel mit anderen Faltungen oder Pooling-Layern zu Inception-Modulen kombinieren.

</div>

In [ ]:
import torch
import torch.nn as nn
import lightning as L

# Definition des Inception-Moduls als LightningModule
class InceptionModule(L.LightningModule):
    def __init__(self, input_channels=3):
        super().__init__()

        # Zweig A: 1x Conv2D
        self.branch_a = nn.Conv2d(input_channels, 128, kernel_size=1, stride=2, padding=0)

        # Zweig B: 2x Conv2D
        self.branch_b1 = nn.Conv2d(input_channels, 128, kernel_size=1, stride=1, padding=0)
        self.branch_b2 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)

        # Zweig C: AveragePooling + Conv2D
        self.branch_c_pool = nn.AvgPool2d(kernel_size=3, stride=2, padding=1)
        self.branch_c_conv = nn.Conv2d(input_channels, 128, kernel_size=3, stride=1, padding=1)

        # Zweig D: 2x Conv2D
        self.branch_d1 = nn.Conv2d(input_channels, 128, kernel_size=1, stride=1, padding=0)
        self.branch_d2 = nn.Conv2d(128, 128, kernel_size=3, stride=2, padding=1)

        self.relu = nn.ReLU()

    def forward(self, x):
        # Vorwärtsdurchlauf durch Zweig A
        branch_a = self.relu(self.branch_a(x))

        # Vorwärtsdurchlauf durch Zweig B
        branch_b = self.relu(self.branch_b1(x))
        branch_b = self.relu(self.branch_b2(branch_b))

        # Vorwärtsdurchlauf durch Zweig C
        branch_c = self.branch_c_pool(x)
        branch_c = self.relu(self.branch_c_conv(branch_c))

        # Vorwärtsdurchlauf durch Zweig D
        branch_d = self.relu(self.branch_d1(x))
        branch_d = self.relu(self.branch_d2(branch_d))

        # Konkatenieren entlang der Kanalachse (dim=1)
        output = torch.cat([branch_a, branch_b, branch_c, branch_d], dim=1)
        return output

# Modell zusammenbauen und testen
model = InceptionModule()
sample_input = torch.randn(1, 3, 224, 224)  # Beispiel-Eingabe
output = model(sample_input)
print(f"Output Shape: {output.shape}")


In [ ]:
from torchinfo import summary

summary(model, input_size=(1, 3, 224, 224))


In [ ]:
# speichere Modell in Bild
from torchviz import make_dot

sample_input = torch.randn(1, 3, 224, 224).to(
    next(model.parameters()).device
)  # Beispiel-Eingabe
output = model(sample_input)
dot = make_dot(output, params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("InceptionModule_architecture", format="png")
print("Inception Module Architektur gespeichert.")


In [ ]:
display(Image(filename='InceptionModule_architecture.png'))


⚡ Inception-Architektur in PyTorch Lightning (L)
- Die vollständige Inception-Architektur steht in PyTorch als ``torchvision.models.inception_v3`` zur Verfügung, inklusive der mit der ImageNet-Datenmenge vortrainierten Gewichtungen ``(weights="IMAGENET1K_V1")``.
- Ein eng verwandtes Modell ist Xception, das als erweiterte Version der Inception-Architektur gilt. In PyTorch ist Xception5 zwar nicht direkt unter ``torchvision.models`` verfügbar, jedoch kann es über externe Bibliotheken (z.B. timm) geladen werden.
- ⚡ Xception steht für Extreme Inception und ist eine CNN-Architektur, die die Grundidee von Inception radikal weiterentwickelt. Der zentrale Gedanke: die Trennung von räumlichen und kanalbezogenen Merkmalen beim Erlernen von Repräsentationen.

## 7.4 Residuale Verbindungen (ResNets) in Lightning (L)
Residuale Verbindungen sind essenzielle Bausteine moderner, graphenartiger neuronaler Netzwerke und kommen in vielen Architekturen ab 2015 zum Einsatz – unter anderem in Xception, ResNet, EfficientNet und anderen. Die grundlegende Idee wurde erstmals von He et al. (2015) vorgestellt und gewann den ILSVRC-ImageNet-Wettbewerb.

Zwei zentrale Probleme tiefer Netzwerke:
- Vanishing Gradients – bei zunehmender Tiefe verschwindet der Gradient, was das Training erschwert.
- Repräsentations-Engpässe – tiefe Netze können Schwierigkeiten haben, Informationen effizient weiterzugeben.

Lösung mit Residualen Verbindungen (L-style):
- Residuale Verbindungen schaffen eine direkte Abkürzung (Skip Connection) zwischen nicht-benachbarten Layers.
- Statt nur der normalen Weiterverarbeitung wird die Eingabe direkt zur Ausgabe addiert.
- Voraussetzung: Beide Tensoren (Input und Output) müssen die gleiche Shape haben.
- Falls nicht: → Lineare Projektion, z. B. via nn.Conv2d(1x1) oder nn.Linear (ohne Aktivierung), um Formate anzugleichen.

[He et al., Deep Residual Learning for Image Recognition](https://arxiv.org/abs/1512.03385).

In [ ]:
import torch
import torch.nn as nn
import lightning as L


class ResidualBlock(L.LightningModule):
    def __init__(self):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = x
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.relu(self.conv3(out))
        out += residual
        return out


class Model(L.LightningModule):
    def __init__(self):
        super(Model, self).__init__()
        self.residual_block = ResidualBlock()

    def forward(self, x):
        return self.residual_block(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        return loss


# Beispiel für die Modellzusammenfassung
model = Model()
print(model)

# speichere das modell in Bild
from torchviz import make_dot

sample_input = torch.randn(1, 128, 32, 32)  # Beispiel-Eingabe
output = model(sample_input)
dot = make_dot(output, params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("ResidualBlock_architecture_matching", format="png")
print("Residual Block Architektur gespeichert.")


In [ ]:
display(Image(filename='ResidualBlock_architecture_matching.png'))


In [ ]:
class ResidualBlock(L.LightningModule):
    def __init__(self):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(256, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv_residual = nn.Conv2d(256, 128, kernel_size=1, stride=2, padding=0)
        self.relu = nn.ReLU()

    def forward(self, x):
        residual = self.conv_residual(x)
        out = self.relu(self.conv1(x))
        out = self.relu(self.conv2(out))
        out = self.maxpool(out)
        out += residual
        return out


class Model(L.LightningModule):
    def __init__(self):
        super(Model, self).__init__()
        self.residual_block = ResidualBlock()

    def forward(self, x):
        return self.residual_block(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.mse_loss(y_hat, y)
        return loss


# Beispiel für die Modellzusammenfassung
model = Model()
print(model)
# speichere das modell in Bild
from torchviz import make_dot

sample_input = torch.randn(1, 256, 32, 32)  # Beispiel-Eingabe
output = model(sample_input)
dot = make_dot(output, params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("ResidualBlock_architecture_no_matching", format="png")


In [ ]:
display(Image(filename='ResidualBlock_architecture_matching.png'))


Zusammenfassung:

### Dimensionen:
- Der erste Block reduziert die Kanalanzahl von 256 auf 128 und halbiert die räumliche Auflösung durch MaxPooling.
- Der zweite Block behält Kanalanzahl und räumliche Auflösung bei (alles bleibt 128 x 32 x 32).

### Residual-Pfad (Shortcut):
- Der erste Block passt den Shortcut mit einer 1x1 Convolution (stride=2) an die kleinere Output-Größe an.
- Der zweite Block verwendet den Shortcut als Identität, da die Dimensionen gleich bleiben.

### Downsampling:
- Der erste Block führt Downsampling im Hauptpfad und im Shortcut durch.
- Der zweite Block hat kein Downsampling.

### Komplexität:
- Der erste Block ist kürzer (2 Convs + Pooling), aber verändert die Dimensionen.
- Der zweite Block ist tiefer (3 Convs) und arbeitet auf gleicher Dimension.

### Einsatz:
- Der erste Block ist für Übergänge zwischen Netzwerkschichten gedacht.
- Der zweite Block eignet sich für tiefe Verarbeitung innerhalb derselben Schicht.

<div class="alert alert-block alert-info">

### Engpässe der Repräsentation beim Deep Learning

Bei einem `Sequential`-Modell baut jeder Layer auf dem vorhergehenden Layer
auf und hat deshalb nur Zugriff auf die in den Aktivierungen der vorhergehenden
Layer enthalten Informationen. Sollte einer der Layer zu klein sein (beispielsweise
weil er nur niedrigdimensionale Merkmale besitzt), ist das Modell
auf die Menge der Informationen beschränkt, die in die Aktivierungen dieses
Layers gepackt werden können.
    
Dieses Konzept lässt sich durch eine Analogie zu einer Signalverarbeitung besser
verstehen. Stellen Sie sich eine Pipeline zur Verarbeitung von Audiodaten vor, bei
der eine Reihe von Operationen jeweils die Ausgabe der vorhergehenden Operation
als Eingabe erhält. Wenn nun eine der Operationen das Signal so beschneidet,
dass nur noch niedrige Frequenzen (z.B. von 0 bis 15 kHz) verbleiben,
können die nachfolgenden Operationen die entfernten Frequenzen nicht wiederherstellen.
Der Informationsverlust ist dauerhaft. Residuale Verbindungen, die
früher vorhandene Informationen nachfolgenden Operationen wieder zur Verfügung
stellen, lösen dieses Problem von Deep-Learning-Modellen teilweise.
    
</div>
<div class="alert alert-block alert-info">

### Vanishing Gradients beim Deep Learning

Der wichtigste Algorithmus zum Trainieren von DNNs, die Backpropagation,
funktioniert folgendermassen: Ein Feedback-Signal der Ausgabe wird an tiefer
gelegene Layer weitergeleitet. Wenn dieses Signal einen großen Stapel von Layern
durchläuft, kann es sehr schwach werden oder sogar ganz verloren gehen –
das NN kann dann nicht trainiert werden. Man spricht hier vom Problem des verschwindenden
Gradienten.
    
Dieses Problem tritt sowohl bei DNNs als auch bei RNNs mit sehr grossen
Sequenzen auf. In beiden Fällen muss das Feedback-Signal eine lange Reihe von
Operationen durchlaufen. Das vom LSTM-Layer genutzte Verfahren zur Behebung
dieses Problems in RNNs ist Ihnen bereits bekannt: Er verwendet eine
Carry-Spur, die Informationen parallel zur eigentlichen Verarbeitung der Daten
weiterleitet. In Feedforward-Netzen funktionieren residuale Verbindungen auf
ähnliche Weise. Sie sind sogar noch einfacher und nutzen eine Carry-Spur, die
rein lineare Informationen parallel zur eigentlichen Verarbeitung der Daten weiterleitet,
und ermöglichen es so, dass sich Gradienten durch beliebig tiefe Stapel
von Layern ausbreiten können.
    
</div>

## 7.5 Geteilte Gewichte: Symmetrie nutzen!
Eine wichtige Eigenschaft moderner Deep-Learning-Frameworks wie Lightning ist die Möglichkeit, eine Instanz eines Layers mehrfach zu verwenden, ohne dass neue Gewichte erzeugt werden. Das bedeutet: Ein Layer wird nur einmal instanziiert und bei wiederholtem Aufruf mit denselben Gewichten verwendet.

Vorteile:
- Man kann gemeinsam nutzbare Zweige eines Netzwerks bauen, die dieselben Repräsentationen erzeugen.
- Die Gewichte werden geteilt und gleichzeitig anhand beider Eingaben gelernt.
- Das Modell kann z. B. Ähnlichkeiten zwischen Eingaben erkennen, ohne Redundanz bei der Berechnung.

Beispielanwendung:
Man möchte beurteilen, wie ähnlich sich zwei Sätze inhaltlich sind (z. B. für Duplikaterkennung in Texten). Dafür werden zwei Eingaben (Satz A und Satz B) in ein gemeinsames LSTM gegeben, die Ausgaben kombiniert und ein Ähnlichkeitswert zwischen 0 und 1 berechnet.

- Satz A und Satz B werden symmetrisch verarbeitet.
- Ein gemeinsam genutztes LSTM verarbeitet beide Sätze.
- Solche Modelle heißen Siamese LSTM oder Modelle mit geteilten Gewichten.

In [ ]:
import torch
import torch.nn as nn
import pytorch_lightning as pl


class SiameseLSTM(pl.LightningModule):
    def __init__(self):
        super(SiameseLSTM, self).__init__()
        # Ein LSTM-Layer wird einmal instanziiert und auf beide Eingaben angewendet
        self.lstm = nn.LSTM(input_size=128, hidden_size=64, batch_first=True)
        self.fc = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, left_input, right_input):
        # Gemeinsames LSTM für beide Eingaben
        left_output, _ = self.lstm(left_input)
        right_output, _ = self.lstm(right_input)

        # Nur den letzten Zeitschritt extrahieren (für Klassifikation)
        left_out_final = left_output[:, -1, :]
        right_out_final = right_output[:, -1, :]

        # Verkettung beider Repräsentationen
        merged = torch.cat([left_out_final, right_out_final], dim=1)

        # Vorhersage Ähnlichkeit (zwischen 0 und 1)
        similarity = self.sigmoid(self.fc(merged))
        return similarity

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-3)

    def training_step(self, batch, batch_idx):
        left_input, right_input, targets = batch
        preds = self(left_input, right_input)
        loss = nn.functional.binary_cross_entropy(preds, targets)
        return loss


# Modell initialisieren und Beispiel-Durchlauf
model = SiameseLSTM()
x_left = torch.randn(4, 10, 128)  # (Batch, SeqLen, InputDim)
x_right = torch.randn(4, 10, 128)
output = model(x_left, x_right)
print(output.shape)  # Erwartet: [4, 1]

# Visualisierung (ähnlich Keras plot_model)
from torchviz import make_dot

dot = make_dot(output, params=dict(model.named_parameters()))
dot.graph_attr.update(dpi="600")
dot.render("SiameseLSTM_architecture", format="png")
print("Architektur gespeichert als PNG.")


In [ ]:
display(Image(filename='SiameseLSTM_architecture.png'))


## 7.7 Modelle als Layer 
Ein wichtiger Punkt: Mit der Lightning-API bzw. PyTorch können Sie Modelle genauso wie Layer verwenden – tatsächlich lässt sich ein Modell als ein »grösserer Layer« betrachten.

Dies gilt sowohl für ``nn.Sequential`` als auch für individuell definierte Klassen, die von nn.Module oder ``L.LightningModule`` erben. Sie können ein solches Modell mit einem Eingabetensor aufrufen und erhalten direkt einen Ausgabetensor zurück:

In [ ]:
import torch
import torch.nn as nn
import lightning as L


# Einfaches Modell als Layer
class SimpleModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 32),
            nn.ReLU(),
            nn.Linear(32, 10),
            nn.Softmax(dim=1),
        )

    def forward(self, x):
        return self.model(x)


# Nutzung des Modells als Layer
class ModelAsLayer(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.model = SimpleModel()

    def forward(self, x):
        return self.model(x)


# Check if GPU is available and use it if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_layer = ModelAsLayer().to(device)
print(model_layer)
# torchsummary
from torchsummary import summary

summary(model_layer, (32,), device=str(device))


Ein Modell, das die **Bilder einer Stereokamera als Eingabe** verwendet, ist ein einfaches
praktisches Beispiel für die Wiederverwendung von Modellinstanzen: 
- zwei parallel ausgerichtete Kameras mit nur wenigen Zentimetern Abstand. 
- Ein solches Modell kann räumliche Tiefe wahrnehmen, was sich für viele Anwendungen als nützlich erweist. 
- Es sollte nicht notwendig sein, zwei unabhängige Modelle zum Extrahieren der visuellen Merkmale der linken und der rechten Kamera zu verwenden, bevor sie zusammengeführt werden. 
- Zur Verarbeitung auf dieser Ebene können beide Eingaben gemeinsam genutzt werden, und zwar durch Layer, die dieselben Gewichtungen verwenden und daher auch die Repräsentationen gemeinsam haben.

Und so können Sie ein Modell mit gemeinsamer Faltungsbasis in Keras implementieren:

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import lightning as L
from torchsummary import summary
from torchviz import make_dot


# Feature Extractor (EfficientNet als Ersatz für Xception)
class FeatureExtractor(L.LightningModule):
    def __init__(self):
        super().__init__()
        base_model = models.efficientnet_b0(pretrained=False)
        self.features = nn.Sequential(
            *list(base_model.children())[:-1]
        )  # Remove classifier

    def forward(self, x):
        return self.features(x)  # Output: [B, 1280, 8, 8] bei 250x250 Input


# Stereo Camera Model
class StereoCamModel(L.LightningModule):
    def __init__(self):
        super().__init__()
        self.extractor = FeatureExtractor()

    def forward(self, left_img, right_img):
        left_feat = self.extractor(left_img)  # [B, 1280, 8, 8]
        right_feat = self.extractor(right_img)  # [B, 1280, 8, 8]
        merged = torch.cat([left_feat, right_feat], dim=1)  # [B, 2560, 8, 8]
        return merged


# Instantiate model
model = StereoCamModel()
print(model)


# torchsummary – benötigt die Module in nn.Module, daher die inneren Features zeigen
# Trick: Dummy-Modul für torchsummary
class StereoSummaryWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, left_img, right_img):
        return self.model(left_img, right_img)


summary_model = StereoSummaryWrapper(model)

# Zeige Summary
summary(summary_model, [(3, 250, 250), (3, 250, 250)], device="cpu")

# Torchviz: Computational Graph anzeigen
left = torch.randn(1, 3, 250, 250)
right = torch.randn(1, 3, 250, 250)
output = model(left, right)

# Visualisiere den Graphen
dot = make_dot(output, params=dict(model.named_parameters()))
dot.format = "png"
dot.render("stereo_cam_model_graph", format="png")  # Speichert stereo_cam_model_graph.png


In [ ]:
display(Image(filename='stereo_cam_model_graph.png'))
